In [17]:
"""
Generating data for "Predicting Forest Regrowth in Amazonia given Land Use History"
Author: Ana Catarina Avila
Date: Dec 2023
"""

import ee
import geemap
#from geetools import batchvamo
# import pandas as pd
import numpy as np
# from osgeo import gdal
# import os
# import datetime

# Authenticate to Earth Engine
try:
  ee.Initialize()
except Exception as e:
  ee.Authenticate()
  ee.Initialize(project='ee-ana-zonia')


In [18]:
# Switches
region = "br_amazon"

# Choose year range
first_year = 1985
last_year = 2020 # year of biomass data

# how many meters around a secondary forest is considered a "neighborhood" for mature forest calculations
mat_neighbor = 1000

plot_region = ee.Geometry.Rectangle([-47, -3.5, -46, -2.5])
Map = geemap.Map()
Map.center_object(plot_region, zoom=9)

In [19]:
if region == "br_amazon":
    roi = ee.FeatureCollection("projects/ee-ana-zonia/assets/br_biomes").filter(ee.Filter.eq("id", 18413)).geometry()
elif region == "br":
    roi = ee.FeatureCollection("projects/ee-ana-zonia/assets/br_shapefile").geometry()
else:
    # land use for the pan amazonian dataset (spanning all amazonian countries)
    lulc = ee.Image("projects/mapbiomas-raisg/public/collection1/mapbiomas_raisg_panamazonia_collection1_integration_v1").byte()
    # note: there's less land use categories here than for the Brazilian territory.

# Load the images and feature collections
if not region == "panamaz":
  lulc = (ee.Image("projects/mapbiomas-workspace/public/collection8/mapbiomas_collection80_integration_v1")
      .clip(roi)
      .select([f"classification_{year}" for year in range(first_year, last_year+1)])).byte()
  fire = (ee.Image("projects/mapbiomas-workspace/public/collection7_1/mapbiomas-fire-collection2-annual-burned-coverage-1")
      .clip(roi)
      .select([f"burned_coverage_{year}" for year in range(first_year, last_year)])).byte()

## Categorical Variables
Indigenous land, ecoregion, and soil type.

In [25]:
ecoregions = (ee.FeatureCollection("RESOLVE/ECOREGIONS/2017").filterBounds(roi)
                .map(lambda feature: feature.intersection(roi)))
ecoregions_img = ecoregions.reduceToImage(['ECO_ID'], ee.Reducer.first()).rename('ecoreg')

indig_land = ee.FeatureCollection("projects/ee-ana-zonia/assets/indig_land").filterBounds(roi)
indig_land_img = ee.Image().byte().paint(indig_land, 'gid').rename("indig").mask()

Soil categories are strings, which Image objects can't handle. To transform the feature collection into an image, we convert the strings to integers by creating a dictionary.

In [26]:
soil = (ee.FeatureCollection('projects/ee-ana-zonia/assets/DSMW').filterBounds(roi)
                .map(lambda feature: feature.intersection(roi)))

unique_domsoi = soil.aggregate_array('DOMSOI').distinct()

domsoi_dict = ee.Dictionary.fromLists(unique_domsoi, ee.List.sequence(1, unique_domsoi.length()))

soil = soil.remap(unique_domsoi, domsoi_dict.values(), 'DOMSOI')

soil_img = soil.reduceToImage(['DOMSOI'], ee.Reducer.first()).rename('soil')

## Biomass Edge Pixels
Get mean biomass for edge pixels

In [27]:
biomass = ee.Image("projects/ee-ana-zonia/assets/biomass_2020").clip(roi)

# Reproject to 10m
biomass = biomass.reproject(crs=age.projection(), scale=10)
# Reaggregate to 30m (mean value)
biomass = biomass.reduceResolution(reducer=ee.Reducer.mean()).reproject(crs=age.projection())
# Mask only to regions with age greater than zero (secondary forests)
biomass = biomass.updateMask(age).float().rename('agbd')

## Mature Forests

In [28]:
mature_mask = lulc.eq(3)
# Mask the image to keep only pixels with the value 3 in all bands
mature_mask = mature_mask.reduce(ee.Reducer.allNonZero())
mature_biomass = biomass.updateMask(mature_mask).rename('mat_biomass')

# # Compute the median biomass values for mature forest per ecoregion.
median_mature = mature_biomass.reduceRegions(ecoregions, reducer=ee.Reducer.median(), scale = 30, crs = mature_biomass.projection().crs())
# Convert the FeatureCollection to an image.
median_mature = median_mature.reduceToImage(['mat_biomass'], ee.Reducer.first())

# Compute sum over round kernel
# assuming that 99.7% of seeds come from forests within mat_neighbor distance, that's 3 standard deviations
# sd = mat_neighbor/3
# kernel = ee.Kernel.gaussian(radius = mat_neighbor, sigma = sd, units='meters', normalize = True)
# weighed_sum_mature = mature_biomass.convolve(kernel).rename('weighted_sum_mature')
# weighed_sum_mature

projects/ee-ana-zonia/assets/frag_2020


## Land use and land cover

In [29]:
#   last observed land use type - given secondary ages, get
# if age is 1 in 2020, it was not forest in 2019
# which means I want category in 2019
# if age is 35 in 2020, it was not forest in 1985
# which means I want category in 1985
lulc_masked = lulc.updateMask(age)

last_LU = ee.Image().constant(0).byte().rename('last_LU')

for i in range(first_year + 1, last_year):  # 1986-2020
  year = f'classification_{i}'
  age_mask = age.eq(last_year-i); #keep only the pixels with age equivalent to the correspondent year
  last_LU_observation = lulc.select(year).updateMask(age_mask) #keeps only land use classes of the year before abandonment
  last_LU = last_LU.add(last_LU_observation)

In [30]:
LU_sum = ee.Image().byte()

for val in LU_index:
  lulc_val = lulc_masked.eq(val)
  lulc_val_mask = lulc_val.mask()
  num_cells = lulc_val.reduce(ee.Reducer.sum()).rename(f'lulc_sum_{val}')
  LU_sum = LU_sum.addBands(num_cells)

LU_sum = LU_sum.slice(1).rename('LU_sum')

# Map = geemap.Map(center=[-10, -40], zoom=4)
# vizpar_age = {'min': 1, 'max': last_year - first_year, 'palette': ['blue', 'red']}  # Blue for 0, Red for 1
#vizpar_lulc = {'min': 1, 'max': max(LU_index), 'palette': lulc_palette}
# Map.addLayer(age, vizpar_age, "ages")
# Map.addLayer(num_cells, vizpar_age, "lulc_masked")
# Map
#img_export = img_export + [last_LU, LU_sum]

### Fire
Note that fire has different transform than lulc, so the projection will need to be adjusted when exporting.

In [31]:
# fire has the value of the land use type that burned.
# Transforming into a fire mask:
fire = fire.gt(0)
num_fires = fire.reduce(ee.Reducer.sum()).rename('num_fires')

# get fire frequency data from Mapbiomas - double check it.
# fire_freq = ee.Image("projects/mapbiomas-workspace/public/collection7_1/mapbiomas-fire-collection2-fire-frequency-1").clip(roi)
# fire_freq = fire_freq.select('fire_frequency_1985_2020')

# how many years ago was each fire? #############################
# Get the number of bands
num_bands = fire.bandNames().size()
# Create a sequence of numbers from 1 to num_bands
years_ago = ee.List.sequence(1, num_bands)
years_ago = years_ago.reverse()

# # # Map over the image and set values based on the band index
constant_images = ee.ImageCollection.fromImages(
    years_ago.map(lambda year: ee.Image.constant(year))).toBands()

time_since_all_fires = fire.multiply(constant_images)

old_names = time_since_all_fires.bandNames().getInfo()
new_names = [name.replace('burned_coverage', 'time_since_fire') for name in old_names]
time_since_all_fires = time_since_all_fires.select(old_names).rename(new_names)

# how many years ago was the LAST fire? #############################
last_fire = time_since_all_fires.reduce(ee.Reducer.lastNonNull()).rename('last_fire')

# Map = geemap.Map(center=[-10, -40], zoom=4)
# Map.addLayer(num_fires, {'min':0, 'max':35}, 'num_fires')
# Map

# img_export = img_export + [num_fires, time_since_all_fires, last_fire]

## Climate

### CWD

Calculate mean AET from MODIS and calculate CWD from the precipitation values as in Celso

Since MODIS only goes back to 2000, for now we are stuck with a fixed value for ET


In [113]:
# Function to calculate mean AET and add year property
# select for mature forests since the values can be put off by deforestation (causes lower ET)

start = "2002-01-01"
end = "2020-12-01"

modis = ee.ImageCollection("MODIS/061/MOD16A2GF") \
         .filterDate(start, end) \
         .select('ET', 'ET_QC').map(lambda image: image.clip(roi).updateMask(mature_mask)
         .reproject(scale = 10000, crs = 'EPSG:4326').reduceResolution(ee.Reducer.median()))

months = ee.List.sequence(1, 12)
years = ee.List.sequence(1985, 2019)

In [114]:
# code sourced from https://spatialthoughts.com/2021/08/19/qa-bands-bitmasks-gee/
def bitwise_extract(input, from_bit, to_bit):
   mask_size = ee.Number(1).add(to_bit).subtract(from_bit)
   mask = ee.Number(1).leftShift(mask_size).subtract(1)
   return input.rightShift(from_bit).bitwiseAnd(mask)

def apply_QA_mask (image):
    QA = image.select('ET_QC')
    ET = image.select('ET').multiply(0.0125)  # multiply by the scale 0.1, divide by 8 to get daily info
    cloud_mask = bitwise_extract(QA, 3, 4).lte(0)
    qa_mask = bitwise_extract(QA, 5, 7).lte(1)
    mask = cloud_mask.And(qa_mask)
    return ET.updateMask(mask).set('system:time_start', image.get('system:time_start'))

# mask quality of pixels
modis_masked = modis.map(apply_QA_mask)

# Loop through the months and filter images
def et_monthly(month):
    eight_day_images = modis_masked.select('ET').filter(ee.Filter.calendarRange(month, month, 'month'))
    monthly_et = eight_day_images.median().multiply(30).reduceRegions(ecoregions, \
                                                                      reducer = ee.Reducer.median(), scale = 10000)
    return monthly_et.set('month', month)

monthly_et_list = ee.List(months.map(et_monthly))

### Terraclim and seasonality

Bring temperature and precipitation and calculate seasonality

In [115]:
terraclim = ee.ImageCollection('IDAHO_EPSCOR/TERRACLIMATE') \
              .filterDate('1985-01-01', '2019-12-31') \
              .map(lambda image : image.clip(roi) \
                  .reproject(scale = 10000, crs = 'EPSG:4326').reduceResolution(ee.Reducer.median()))

maxtemp = terraclim.select('tmmx').map(lambda image: image.multiply(0.1))
mintemp = terraclim.select('tmmn').map(lambda image: image.multiply(0.1))
radiation = terraclim.select('srad').map(lambda image: image.multiply(0.1))
prec = terraclim.select('pr')

In [110]:
monthly_wd = prec.map(lambda image: image.subtract(100))
# monthly_wd_list = monthly_wd.toList(monthly_wd.size())



def cwd_monthly(current_image, previous_dict):
    current_image = ee.Image(current_image)
    previous_dict = ee.Dictionary(previous_dict)
    previous_image = ee.Image(previous_dict.get('result_image'))
    result_image = previous_image.add(current_image).where(previous_image.gt(0), 0) \
                                   .set('system:time_start', current_image.get('system:time_start'))
    return ee.Dictionary({
        'result_image': ee.Image(result_image),
        'cwd_list': ee.List(previous_dict.get('cwd_list')).add(result_image)})

# Use iterate to apply the function to each image
cwd_dict = ee.Dictionary(water_deficit.iterate(cwd_monthly, ee.Dictionary({'result_image': ee.Image(0),
                                                               'cwd_list': ee.List([])})))
cwd_list = ee.List(cwd_dict.get('cwd_list'))


def mcwd_yearly(year):
    # Calculate the start and end indices for the current year
    start = ee.Number(year).subtract(1985).multiply(12)
    end = start.add(12)
    # Slice the list to get the images for the current year
    yr = ee.List(cwd_list).slice(start, end)
    cwd_a = ee.ImageCollection.fromImages(yr)
    mcwd_a = cwd_a.reduce(ee.Reducer.min())
    return mcwd_a.set('year', year)

# Map the function over the range
annual_mcwd = ee.ImageCollection.fromImages(years.map(mcwd_yearly))

In [98]:
def seasonality_calc(y):
    yr_prec = prec.filter(ee.Filter.calendarRange(y, y, 'year'))
    # get mean monthly precipitation for the year
    mean_prec = yr_prec.reduce(ee.Reducer.mean())
    # Calculate absolute deviations of monthly precipitation from mean
    deviations = yr_prec.map(lambda month:month.subtract(mean_prec).abs())
    # Calculate sum of absolute deviations for each month
    sum_deviations = deviations.reduce(ee.Reducer.sum())
    # Calculate total annual precipitation
    total_annual_prec = yr_prec.reduce(ee.Reducer.sum())
    # Calculate Seasonality Index (SI)
    seasonality_index = sum_deviations.divide(total_annual_prec)
    return seasonality_index

yearly_SI = ee.ImageCollection.fromImages(years.map(seasonality_calc))

In [102]:
def meantemp_calc(y):
    # Filter both maxtemp and mintemp ImageCollections by year at once
    yearly_maxtemp = maxtemp.filter(ee.Filter.calendarRange(y, y, 'year'))
    yearly_mintemp = mintemp.filter(ee.Filter.calendarRange(y, y, 'year'))
    # Calculate the mean temperature directly
    mean_temp = yearly_maxtemp.reduce(ee.Reducer.mean()).add(yearly_mintemp.reduce(ee.Reducer.mean())).divide(2)
    return mean_temp

# Map the function over the years
mean_temp = ee.ImageCollection.fromImages(years.map(meantemp_calc))

## Unify

In [ ]:
# cwd = ee.Image('projects/ee-ana-zonia/assets/cwd_chave').rename('cwd')
# proj = age.projection().getInfo()

#img_export.append(cwd)
# Add the 'ECO_ID' property of ecoregions as a new band in the 'age' image.
img_export = [age.pixelLonLat().float(), biomass, num_fires, last_fire, last_LU, ecoregions_img, indig_land_img]
# for eco_id in ecoregions.aggregate_array("ECO_ID").getInfo():
#     ecoreg =  ecoregions.filter(ee.Filter.eq("ECO_ID",eco_id))

## Export

In [80]:
img_export = age.addBands(biomass).selfMask().float()

fishnet = geemap.fishnet(roi, h_interval=2.0, v_interval=2.0, delta=0.5)
# Set 'id' as a property of the features
fishnet = fishnet.map(lambda feature: feature.set('id', feature.id()))
fishnet_info = fishnet.getInfo()['features']

for feature in fishnet_info:
    id = feature['id']
    grid_cell = fishnet.filter(ee.Filter.eq('id', id))
    img = img_export.clip(grid_cell)
    img = img.sample(grid_cell, scale = 30)
    task = ee.batch.Export.table.toDrive(
    collection=img,
    description=f'{id}_age_agbd',
    folder='drive_export',
    fileFormat='CSV')
    task.start()